In [1]:
!pip install tensorflow h5py pyyaml


In [2]:
import os
import tensorflow as tf
from tensorflow import keras


In [3]:
print(tf.__version__)


2.18.0


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Use a small dataset for quick testing
train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

# Create a new model instance
model = create_model()

# Display model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
checkpoint_path = "model_checkpoints/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback to save model weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

# Train the model with checkpoint callback
model.fit(train_images,
          train_labels,
          epochs=5,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])


Epoch 1/5
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4768 - loss: 1.6358
Epoch 1: saving model to model_checkpoints/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.4941 - loss: 1.5929 - val_accuracy: 0.7870 - val_loss: 0.7157
Epoch 2/5
25/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8948 - loss: 0.4446
Epoch 2: saving model to model_checkpoints/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8936 - loss: 0.4398 - val_accuracy: 0.8320 - val_loss: 0.5320
Epoch 3/5
27/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9286 - loss: 0.3012
Epoch 3: saving model to model_checkpoints/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9276 - loss: 0.3020 - val_accuracy: 0.8620 - val_loss: 0.4740
Epoch 4/5
25/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9276 - loss: 0.2267
Epoch 4: saving model to model_checkpoints/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9302 - loss: 0.2248 - val_accuracy: 0.

In [8]:
# Create a new model instance
new_model = create_model()

# Load the saved weights
new_model.load_weights("model_checkpoints/cp.weights.h5")

# Evaluate the model on test data
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model accuracy: {:5.2f}%".format(100 * acc))


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 - 0s - 11ms/step - accuracy: 0.8540 - loss: 0.4469
Restored model accuracy: 85.40%


In [9]:
# Save the entire model
model.save("saved_model.keras")

# Load the model back
reloaded_model = tf.keras.models.load_model("saved_model.keras")

# Verify performance
loss, acc = reloaded_model.evaluate(test_images, test_labels, verbose=2)
print("Reloaded model accuracy: {:5.2f}%".format(100 * acc))


32/32 - 1s - 16ms/step - accuracy: 0.8540 - loss: 0.4469
Reloaded model accuracy: 85.40%
